In [2]:
%matplotlib inline
import seaborn as sns; 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib import gridspec
import scipy
style.use('ggplot')
import datetime
from io import StringIO
import re
import os
import numpy as np
from datetime import datetime
import apt_pkg
apt_pkg.init_system()

# Load installed packages

In [3]:
packages = pd.read_csv('../../data/for_analysis/installed_packages.csv',
                      usecols=['package','version','archive','last_updated','date'])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
packages.drop_duplicates(inplace=True)

In [5]:
packages.shape

(2303862, 5)

In [6]:
packages[['package','version','last_updated']].drop_duplicates().shape

(2257988, 3)

In [6]:
for col in packages.columns:
    packages[col] = packages[col].apply(str)

In [7]:
packages.head(2)

,package,version,last_updated,archive,date
0,0ad,0.0.21-2,2018-12-10,stretch,2016-11-19
1,0ad-data,0.0.21-1,2018-12-10,stretch,2016-11-19


# Load debian packages

In [6]:
debian_packages = pd.read_csv('../../data/prepared_data/debian_packages.csv',
                             usecols=['package','version','archive','date'])
debian_packages.drop_duplicates(inplace=True)
debian_packages.shape

(954495, 4)

In [7]:
for col in debian_packages.columns:
    debian_packages[col] = debian_packages[col].apply(str)

In [10]:
debian_packages.rename(columns={'version':'version_c','date':'date_c'}, inplace=True)

In [12]:
debian_packages.tail(2)

,package,version_c,archive,date_c
954493,zzuf,0.15-1+b1,buster,2017-06-19
954494,zzuf,0.15-1+b1,stretch,2017-03-06


In [11]:
packages = (packages
            .merge(debian_packages,
                   on = ['package','archive'],
                   how = 'left'
                  )
           )
packages.shape

(26614399, 7)

In [12]:
# Remove package version that were created after the containers inspection
packages = packages.query('last_updated>=date_c').copy()
packages.shape

(24367357, 7)

In [16]:
packages.head(2)

,package,version,last_updated,archive,date,version_c,date_c
0,0ad,0.0.21-2,2018-12-10,stretch,2016-11-19,0.0.17-1,2015-04-26
1,0ad,0.0.21-2,2018-12-10,stretch,2016-11-19,0.0.18-1,2015-04-27


# Let's compute the Time lag using latest available release

In [14]:
time_lag = packages.query('date_c >= date')
time_lag.shape

(3038690, 7)

In [15]:
time_lag = (time_lag
            .sort_values('date_c', ascending=False)
            .groupby(['package','version','archive','last_updated'])
            .first()
            .reset_index()
           )
time_lag.shape

(2262749, 7)

In [16]:
time_lag['date'] = pd.to_datetime(time_lag['date'])
time_lag['date_c'] = pd.to_datetime(time_lag['date_c'])

In [17]:
time_lag['tlag']=time_lag.apply(lambda d: (d['date_c'] - d['date']).days, axis=1)

In [19]:
time_lag.rename(columns={'version_c':'ideal'}, inplace=True)

In [20]:
time_lag = time_lag[['package','version','archive','last_updated','ideal','tlag']]

In [22]:
time_lag['compare'] = time_lag.apply(lambda d: compare(d['version'], d['ideal']), axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Let's compute the Version lag using latest available release

In [ ]:
def compare(version,compare_to):
    return apt_pkg.version_compare(str(version), str(compare_to))
    # 1 if version > compare_to
        
packages['compare'] = packages.apply(lambda d: compare(d['version'], d['version_c']), axis=1)
packages.shape

In [26]:
packages.compare.unique()

array([   1,    0,    2,    3,    8,    4,    5,   -1,   -2,   -3,   -4,
          6,   -5,    7,  303,  -16,   16, -303,  -17,   17,   -6,   -7,
         -8,  300,    9,   82,  201,   98,  -98,  184,   11,   -9,  188,
        199,  103,  104,  220, -114, -300,  114,  -14,   15,   14,   13,
         12,   10,   21,   97,  200])

In [27]:
version_lag = packages.query('compare < 0')
version_lag.shape

(783735, 8)

In [28]:
version_lag = (version_lag[['package','version','archive','last_updated','compare']]
               .groupby(['package','version','archive','last_updated'])
               .count()
               .reset_index()
               .rename(columns = {'compare':'vlag'})
              )
version_lag.shape

(374725, 5)

# Merge vlag and tlag into one dataframe

In [33]:
time_lag = (time_lag
            .merge(version_lag,
                  on =['package','version','archive','last_updated'],
                   how = 'left'
                  )
           )
time_lag.fillna(0, inplace=True)
time_lag.shape

(2262749, 7)

In [39]:
time_lag = time_lag.query('vlag==tlag or vlag != 0')

In [40]:
time_lag.to_csv('../../data/for_analysis/tv_lag_ideal.csv', index=False)